<h1><b font size = "50px" font-color: black>Spam Detecion Filter<b></h1>

By using the dataset from https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection where they collected spam and ham messages of over 5,000+ SMS messages, we can use basic machine learning from these data to estimate if a message is spam or ham

In [38]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

Importing the basic modules for Data Science

In [39]:
df = pd.read_csv("./spam.csv", encoding='latin-1')
clean = df.drop(['Unnamed: 2'], axis=1)
clean = clean.drop(['Unnamed: 3'], axis = 1)
clean = clean.drop(['Unnamed: 4'], axis = 1)
clean.rename(columns = {'v1' : 'Category', 'v2' : 'Message'}, inplace=True)
clean["Message"] = clean["Message"].str.replace('[^a-zA-Z]',' ', regex = True)
clean["Message"] = clean["Message"].str.lower()

Getting the Database read, titling the unnamed column titles, making everything lower case for easier identification and removing all non letters, we cleaned up the data to be more easily read and analyzed

In [40]:
clean_random = clean.sample(frac=1, random_state=1)
train_data = clean_random[:round(len(clean_random)*0.75)].reset_index(drop=True)
test_data = clean_random[round(len(clean_random)*0.75):].reset_index(drop=True)

We seperated the data into 2 sections, the training set which we use to train our models for the prediction if a message is spam or ham and the test data where we end up using seperate data to test how accuracte our model is.

In [41]:
train_data['Message'] = train_data['Message'].str.split()
words = []
for i in train_data['Message']:
    for word in i:
        words.append(word)
words = list(set(words))
word_freq = pd.DataFrame(words)
word_freq['#Spam'] = 0
word_freq['#Ham'] = 0
word_freq.rename(columns={word_freq.columns[0]: "Word" }, inplace = True)
for i in range(0,len(train_data['Message'])):
    spam_ham = train_data.iloc[i]['Category']
    for word in train_data.iloc[i]['Message']:
        location = word_freq.loc[word_freq['Word'] == word]
        if spam_ham == "spam":
            word_freq.at[location.index[0],'#Spam']+=1
        elif spam_ham == "ham":
            word_freq.at[location.index[0],'#Ham']+=1

Create a seperate database where we count for each seperate word in the training data set the number of times its in a <em>spam</em> message and the number of times its in a <em>ham</em> message.

In [42]:
word_prob = word_freq
total_spam = (train_data["Category"]=="spam").sum()
total_ham = (train_data["Category"]=="ham").sum()
word_prob["P(E|S)"]= (word_prob["#Spam"]+0.5)/(total_spam+1)
word_prob["P(E|¬S)"]= (word_prob["#Ham"]+0.5)/(total_ham+1)
word_prob.drop(columns=["#Spam","#Ham"], axis = 1, inplace=True)